In [70]:
#Supervisor
#Langchain Supervisor

In [143]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

#Initialize Open Ai
from dotenv import load_dotenv

llm=ChatOpenAI(temperature=0,model="gpt-4o-mini")


In [144]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D8E13C05C0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D8E0372AE0>, root_client=<openai.OpenAI object at 0x000001D8E156BB60>, root_async_client=<openai.AsyncOpenAI object at 0x000001D8E13C2D80>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [145]:
#Tool-1
from langchain_community.tools import DuckDuckGoSearchRun

def search_duckduckgo(query: str):
    """Searches DuckDuckGo using LangChain's DuckDuckGoSearchRun tool."""
    search = DuckDuckGoSearchRun()
    return search.invoke(query)


results=search_duckduckgo("who is George Foreman")

print(results)


#Tool2
def add(a:float,b:float) -> float:
    """Add two numbers"""
    return a +b 

#Tool3
def multiply(a:float,b:float)->float:
    """Multiply two numbers"""
    return a*b



George Foreman George Foreman (born January 10, 1949, Marshall, Texas, U.S.—died March 21, 2025) was an American boxer who twice was the world heavyweight champion (1973-74, 1994-95). When Foreman regained the heavyweight title at age 45, he was the oldest world heavyweight champion. (Read Gene Tunney's 1929 Britannica essay on boxing.) Foreman grew up in Houston, Texas, and learned to box in a U.S ... Legendary boxer George Foreman has died at 76, according to a statement released by his family Friday night on his official Instagram account. The cause of death was not initially revealed by the ... In 1994, George Foreman launched the George Foreman Grill, and over 100 million grills have been sold. At the time of his death, Foreman was said to have been worth $300 million, according to ... George Edward Foreman was born on Jan. 10, 1949, in Marshall, Texas. His father, J.D. Foreman, was a railroad construction worker. His mother was Nancy Ree (Nelson) Foreman.


In [146]:
#Making Agent 

math_agent=create_react_agent(
    model=llm,
    tools=[add,multiply],
    name="Math Expert",
    prompt="You are a Math Expert, always use one tool ata a time."

)

In [147]:
research_agent = create_react_agent(
    model=llm,
    tools=[search_duckduckgo],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

In [148]:
#Since Supervisor way of method for Agents

from langgraph_supervisor import create_supervisor
# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=llm,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems , calculations, use math_agent."
    )
)


In [149]:
#Compile and Run
apps=workflow.compile()

result=apps.invoke({
    "messages":[
        {
            "role":"user",
            "content":"what is Generative AI?"
        }
    ]
})

In [150]:
print(result)

{'messages': [HumanMessage(content='what is Generative AI?', additional_kwargs={}, response_metadata={}, id='a6a0b03b-59b3-4657-8f3d-f8e79a619d62'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BP13Twq0ltV6fTg1gkfAixWz', 'function': {'arguments': '{}', 'name': 'transfer_to_research_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 103, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BZYG4dqVw2n7gpcs7tNt3wr1FQ8SJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--696cd365-f315-4608-ac55-b400bfa40bd5-0', tool_calls=[{'name': 'transfer_to_research_expert', 'args': {

In [151]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is Generative AI?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_BP13Twq0ltV6fTg1gkfAixWz)
 Call ID: call_BP13Twq0ltV6fTg1gkfAixWz
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

Generative AI, or generative artificial intelligence, is a subset of AI that focuses on creating new content, such as text, images, music, and videos. It works by learning patterns from existing data and can generate responses based on text prompts. This technology has various applications and can produce creative outputs, but it also comes with challenges and limitations.
================================== 

In [155]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is the weather in delhi today and add 10 to it"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is the weather in delhi today and add 10 to it
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_luKGbCaslarBtpLQylGkv5Vc)
 Call ID: call_luKGbCaslarBtpLQylGkv5Vc
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

Today, Delhi is experiencing extreme heat, with temperatures reaching around 46 degrees Celsius. A red alert has been issued due to the intense heatwave. 

If you add 10 to that temperature, it would be 56 degrees Celsius.
================================== Ai Message ==================================
Name: research_expert

Transferring back to supervisor
Tool Calls:
  transfer_

In [ ]:
#The Expectation is the calculations should be handled by Math Agent and Search by Research Agent.